# 📈 Project: Big Brother (Dual-LSTM Trading System)
**Created by:** Daryl James Padogdog  
**Date:** December 2025  
**Framework:** TensorFlow/Keras & Python

---

### 🧠 Why I Built This: "Stop Guessing, Start Calculating"

I developed this system because standard algorithmic trading often fails due to a single problem: **Noise.**

If you train an AI on **5-minute candles**, it panics at every small drop (Overfitting).  
If you train it on **1-hour candles**, it reacts too slowly and misses the entry (Lagging).

I realized that a single model cannot effectively handle both the *Trend* and the *Entry*. So, I split the "brain" of the system into two distinct neural networks.

### ⚙️ The Architecture

1.  **👁️ Big Brother (The Strategist)**
    * **Input:** 1-Hour Data (168-hour lookback).
    * **Role:** Analyzes the macro trend. It ignores short-term volatility to determine if the market is **Bullish** or **Bearish**.
    * **Logic:** Acts as the primary filter. If Big Brother says "Bearish," Little Brother is forbidden from buying.

2.  **🎯 Little Brother (The Sniper)**
    * **Input:** 5-Minute Data (48-period lookback).
    * **Role:** Finds the precise entry point within the trend.
    * **Logic:** Hunts for micro-reversals and price anomalies, aligned with the macro trend.

3.  **🛡️ Risk & Math Layer**
    * **Dynamic Bias Correction:** A rolling window that calculates the AI's recent error rate and adjusts predictions in real-time.
    * **Monte Carlo Simulation:** A projection engine that bootstraps historical performance to estimate wealth generation for 2026.

---

### 📋 Notebook Roadmap
This notebook is structured into four distinct phases:
1.  **Training Phase:** Building and saving the `big_brother` (1H) and `little_brother` (5M) models.
2.  **Hybrid Backtest:** Running the "Dual-Timeframe" logic on test data to verify win rates.
3.  **2025 Prediction:** AI analysis of current market data (Test Set) to visualize performance.
4.  **Wealth Simulation:** A Monte Carlo projection of account growth over the next 2 years.

> *Note: Ensure `TRAIN_1h.csv`, `TRAIN_5m.csv`, `TEST_1h.csv`, and `TEST_5m.csv` are uploaded before running.*

# FIRST WE TRAIN THE BIG BROTHER
**big_brother_v2.keras**

In [ ]:
# ===========================================
# TRAIN BIG BROTHER MODEL (1H Data)
# ===========================================
# Run this FIRST in a fresh Colab session
# After training completes, download 'big_brother_v2.keras'
# ===========================================

import numpy as np
import pandas as pd
import gc
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Bidirectional,
    Input, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [ ]:
# ===========================================
# CONFIGURATION
# ===========================================
TRAIN_FILE = 'TRAIN_1h.csv'
LOOKBACK = 168  # 1 week of hourly data (7*24)
MODEL_NAME = 'big_brother_v2.keras'

FEATURE_COLS = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Returns', 'EMA_20', 'EMA_50', 'EMA_Diff',
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Width', 'BB_Position', 'ATR', 'Volatility'
]

In [ ]:
# ===========================================
# TECHNICAL INDICATORS
# ===========================================
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    return macd, signal_line, histogram

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    middle = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def add_technical_indicators(df):
    df['Returns'] = df['Close'].pct_change()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_Diff'] = df['EMA_20'] - df['EMA_50']
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Volatility'] = df['Returns'].rolling(window=20).std()
    return df

In [ ]:
# ===========================================
# DATA LOADING
# ===========================================
def load_and_preprocess(filepath):
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, sep=';')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')
    df = df.sort_values('Date').reset_index(drop=True)
    df = add_technical_indicators(df)
    df = df.dropna().reset_index(drop=True)

    # Convert to float32 to save memory
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('float32')

    print(f"Loaded {len(df)} rows")
    return df

In [ ]:
# ===========================================
# MODEL
# ===========================================
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model

In [ ]:
# ===========================================
# MAIN
# ===========================================
if __name__ == "__main__":
    print("=" * 60)
    print("  BIG BROTHER TRAINING (1H Trend Model)")
    print("  Created by: Daryl James Padogdog")
    print("  Purpose:    Analyzes the 1-Hour timeframe to determine the")
    print("              Broad Market Trend (Bullish/Bearish).")
    print("              This acts as the primary filter for the system.")
    print("=" * 60)
    print("\n[INFO] Starting process...")

    # Load data
    df = load_and_preprocess(TRAIN_FILE)

    # Normalize
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = df.copy()
    df_scaled[FEATURE_COLS] = scaler.fit_transform(df[FEATURE_COLS]).astype('float32')

    # Create datasets using tf.data (Streaming)
    # -----------------------------------------
    # Split data indices first (prevent data leakage)
    n = len(df_scaled)
    train_n = int(n * 0.85)

    data_array = df_scaled[FEATURE_COLS].values
    target_array = df_scaled['Close'].values

    # Train Dataset
    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        data=data_array[:-LOOKBACK], # slice end to match target
        targets=target_array[LOOKBACK:], # offset targets by lookback
        sequence_length=LOOKBACK,
        sequence_stride=1,
        shuffle=True,
        batch_size=64,
        start_index=0,
        end_index=train_n
    )

    # Validation Dataset
    val_dataset = tf.keras.utils.timeseries_dataset_from_array(
        data=data_array[:-LOOKBACK],
        targets=target_array[LOOKBACK:],
        sequence_length=LOOKBACK,
        sequence_stride=1,
        shuffle=False,
        batch_size=64,
        start_index=train_n
    )

    print("✓ Datasets created (streaming)")

    # Free dataframe memory
    del df, df_scaled
    gc.collect()

    # Build and train
    model = build_model((LOOKBACK, len(FEATURE_COLS)))
    model.summary()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    ]

    model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=callbacks)

    # Save
    model.save(MODEL_NAME)
    print(f"\n✓ Model saved as '{MODEL_NAME}'")
    print("Download this file and keep it safe!")


# NOW WE WILL TRAIN THE LIITLE BROTHER
**little_brother_v2.keras**

In [ ]:
# ===========================================
# TRAIN LITTLE BROTHER MODEL (5M Data)
# ===========================================
# Run this SECOND in a fresh Colab session
# (After Big Brother is trained and downloaded)
# After training completes, download:
#   - 'little_brother_v2.keras'
#   - 'scaler_small.pkl' (needed for predictions)
# ===========================================

import numpy as np
import pandas as pd
import gc
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Bidirectional,
    Input, GlobalAveragePooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
# ===========================================
# CONFIGURATION
# ===========================================
TRAIN_FILE = 'TRAIN_5m.csv'
LOOKBACK = 48  # 4 hours of 5-min data
MODEL_NAME = 'little_brother_v2.keras'
SCALER_NAME = 'scaler_small.pkl'

FEATURE_COLS = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Returns', 'EMA_20', 'EMA_50', 'EMA_Diff',
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Width', 'BB_Position', 'ATR', 'Volatility'
]

In [ ]:
# ===========================================
# TECHNICAL INDICATORS
# ===========================================
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    return macd, signal_line, histogram

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    middle = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def add_technical_indicators(df):
    df['Returns'] = df['Close'].pct_change()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_Diff'] = df['EMA_20'] - df['EMA_50']
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Volatility'] = df['Returns'].rolling(window=20).std()
    return df


In [ ]:
# ===========================================
# DATA LOADING
# ===========================================
def load_and_preprocess(filepath):
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, sep=';')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')
    df = df.sort_values('Date').reset_index(drop=True)
    df = add_technical_indicators(df)
    df = df.dropna().reset_index(drop=True)

    # Convert to float32 to save memory
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('float32')

    print(f"Loaded {len(df)} rows")
    return df


In [ ]:
# ===========================================
# MODEL
# ===========================================
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.3)(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    return model


In [ ]:
# ===========================================
# MAIN
# ===========================================
if __name__ == "__main__":
    print("=" * 60)
    print("  LITTLE BROTHER TRAINING (5M Entry Model)")
    print("  Created by: Daryl James Padogdog")
    print("  Purpose:    Analyzes the 5-Minute timeframe to find precise")
    print("              entry points aligned with the Big Brother trend.")
    print("=" * 60)
    print("\n[INFO] Starting process...")

    # Load data
    df = load_and_preprocess(TRAIN_FILE)

    # Normalize and SAVE the scaler (needed for predictions later)
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = df.copy()
    df_scaled[FEATURE_COLS] = scaler.fit_transform(df[FEATURE_COLS]).astype('float32')

    # Save scaler for later use
    with open(SCALER_NAME, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"✓ Scaler saved as '{SCALER_NAME}'")

    # Also save the close scaler
    close_scaler = MinMaxScaler(feature_range=(0, 1))
    close_scaler.fit(df[['Close']])
    with open('close_scaler.pkl', 'wb') as f:
        pickle.dump(close_scaler, f)
    print("✓ Close scaler saved as 'close_scaler.pkl'")

    # Create datasets using tf.data (Streaming)
    # -----------------------------------------
    # Split data indices first (prevent data leakage)
    n = len(df_scaled)
    train_n = int(n * 0.85)

    data_array = df_scaled[FEATURE_COLS].values
    target_array = df_scaled['Close'].values

    # Train Dataset
    train_dataset = tf.keras.utils.timeseries_dataset_from_array(
        data=data_array[:-LOOKBACK], # slice end to match target
        targets=target_array[LOOKBACK:], # offset targets by lookback
        sequence_length=LOOKBACK,
        sequence_stride=1,
        shuffle=True,
        batch_size=64,
        start_index=0,
        end_index=train_n
    )

    # Validation Dataset
    val_dataset = tf.keras.utils.timeseries_dataset_from_array(
        data=data_array[:-LOOKBACK],
        targets=target_array[LOOKBACK:],
        sequence_length=LOOKBACK,
        sequence_stride=1,
        shuffle=False,
        batch_size=64,
        start_index=train_n
    )

    print("✓ Datasets created (streaming)")

    # Free dataframe memory (the arrays are now referenced by the dataset)
    del df, df_scaled
    gc.collect()

    # Build and train
    model = build_model((LOOKBACK, len(FEATURE_COLS)))
    model.summary()

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
    ]

    model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=callbacks)

    # Save
    model.save(MODEL_NAME)
    print(f"\n✓ Model saved as '{MODEL_NAME}'")
    print("\nDownload these files:")
    print(f"  - {MODEL_NAME}")
    print(f"  - {SCALER_NAME}")
    print("  - close_scaler.pkl")

# YOU CAN NOW ADD A ALGORTIHM TO THE MODELS WHICH YOU LIKE OR TEST IT USING THIS

In [ ]:
# =============
# HYBRID DUAL-TIMEFRAME DEMO
# =============
# Uses the same logic as Trading_AI_model.ipynb:
# - Big Brother: Determines overall market trend (BULLISH/BEARISH)
# - Little Brother: Confirms precise trade entries
# - Only trades when BOTH timeframes agree
# ===========================================

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =============
# CONFIGURATION
# =============
BIG_BROTHER_MODEL = 'big_brother_v2.keras'
LITTLE_BROTHER_MODEL = 'little_brother_v2.keras'
SCALER_SMALL = 'scaler_small.pkl'
CLOSE_SCALER = 'close_scaler.pkl'

TEST_5M = 'TEST_5m.csv'
TEST_1H = 'TEST_1h.csv'

LOOKBACK_BIG = 168   # 1 week of hourly data
LOOKBACK_SMALL = 48  # 4 hours of 5-min data

FEATURE_COLS = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Returns', 'EMA_20', 'EMA_50', 'EMA_Diff',
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Width', 'BB_Position', 'ATR', 'Volatility'
]

# =============
# TECHNICAL INDICATORS
# =============
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    return macd, signal_line, histogram

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    middle = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def add_technical_indicators(df):
    df['Returns'] = df['Close'].pct_change()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_Diff'] = df['EMA_20'] - df['EMA_50']
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Volatility'] = df['Returns'].rolling(window=20).std()
    return df

def load_and_preprocess(filepath):
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, sep=';')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')
    df = df.sort_values('Date').reset_index(drop=True)
    df = add_technical_indicators(df)
    df = df.dropna().reset_index(drop=True)
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('float32')
    print(f"Loaded {len(df)} rows")
    return df

# =============
# HYBRID BACKTEST LOGIC (Optimized)
# =============
def hybrid_backtest(df_5m, little_predictions, df_1h, big_predictions, lookback_5m, lookback_1h, initial_capital=200.0):
    """
    Backtest with PHASE 2 Optimizations & Paper Trading:
    1. Dynamic Bias Correction.
    2. Stricter Threshold & EMA Filter.
    3. Paper Trading ($200 Start).
    """
    results = {
        'total_signals': 0,
        'confirmed_trades': 0,
        'wins': 0,
        'losses': 0,
        'buy_signals': 0,
        'sell_signals': 0,
        'wait_signals': 0,
        'filtered_signals': 0,
        'initial_balance': initial_capital,
        'final_balance': initial_capital,
        'net_profit': 0.0
    }

    trades = []
    equity_curve = [initial_capital]
    signals_log = []

    # --- Bias Correction Setup ---
    bias_window = []
    BIAS_WINDOW_SIZE = 50

    # --- Filters (PHASE 2) ---
    RSI_BUY_MAX = 70
    RSI_SELL_MIN = 30
    MIN_THRESHOLD = 0.0015

    # Align Data
    df_5m_subset = df_5m.iloc[lookback_5m-1:lookback_5m-1+len(little_predictions)].copy()
    df_5m_subset['Predicted_Raw'] = little_predictions

    df_1h_subset = df_1h.iloc[lookback_1h-1:lookback_1h-1+len(big_predictions)].copy()
    df_1h_subset['Predicted_Raw'] = big_predictions

    # 1H Mapping (Added EMAs for Trend Filter)
    df_1h_subset['Hour'] = df_1h_subset['Date'].dt.floor('H')
    hourly_trend = df_1h_subset.set_index('Hour')[
        ['Close', 'Predicted_Raw', 'EMA_20', 'EMA_50']
    ].to_dict('index')

    current_balance = initial_capital

    # --- MAIN LOOP ---
    for i in range(len(df_5m_subset) - 1):
        row = df_5m_subset.iloc[i]
        current_price = row['Close']
        actual_next = df_5m_subset.iloc[i + 1]['Close']

        # 1. Update Bias
        raw_pred_5m = row['Predicted_Raw']
        current_bias = current_price - raw_pred_5m
        bias_window.append(current_bias)
        if len(bias_window) > BIAS_WINDOW_SIZE:
            bias_window.pop(0)
        avg_bias = sum(bias_window) / len(bias_window)

        pred_5m_corrected = raw_pred_5m + avg_bias

        # 2. Big Brother Trend (STRICTER EMA FILTER)
        hour_key = row['Date'].floor('H')
        trend = "NEUTRAL"

        if hour_key in hourly_trend:
            h_data = hourly_trend[hour_key]
            pred_bullish = (h_data['Predicted_Raw'] - h_data['Close']) > -avg_bias
            ema_bullish = (h_data['Close'] > h_data['EMA_50']) and (h_data['EMA_20'] > h_data['EMA_50'])
            ema_bearish = (h_data['Close'] < h_data['EMA_50']) and (h_data['EMA_20'] < h_data['EMA_50'])

            if pred_bullish and ema_bullish:
                trend = "BULLISH"
            elif not pred_bullish and ema_bearish:
                trend = "BEARISH"

        # 3. Little Brother Entry
        pred_change = (pred_5m_corrected - current_price) / current_price

        entry_signal = "WAIT"
        if pred_change > MIN_THRESHOLD:
            entry_signal = "BUY"
        elif pred_change < -MIN_THRESHOLD:
            entry_signal = "SELL"

        # 4. RSI Filter
        if entry_signal == "BUY" and row['RSI'] > RSI_BUY_MAX:
            entry_signal = "WAIT"
            results['filtered_signals'] += 1
        if entry_signal == "SELL" and row['RSI'] < RSI_SELL_MIN:
            entry_signal = "WAIT"
            results['filtered_signals'] += 1

        # === FINAL DECISION ===
        results['total_signals'] += 1
        final_signal = "WAIT"
        trade_return = 0.0

        if trend == "BULLISH" and entry_signal == "BUY":
            final_signal = "BUY NOW"
            results['buy_signals'] += 1
            results['confirmed_trades'] += 1

            trade_return = (actual_next - current_price) / current_price
            trades.append(trade_return)

            if actual_next > current_price:
                results['wins'] += 1
            else:
                results['losses'] += 1

        elif trend == "BEARISH" and entry_signal == "SELL":
            final_signal = "SELL NOW"
            results['sell_signals'] += 1
            results['confirmed_trades'] += 1

            trade_return = (current_price - actual_next) / current_price
            trades.append(trade_return)

            if actual_next < current_price:
                results['wins'] += 1
            else:
                results['losses'] += 1

        elif entry_signal != "WAIT":
            results['wait_signals'] += 1

        # Update Paper Trading Balance
        if trade_return != 0:
            current_balance = current_balance * (1 + trade_return)

        equity_curve.append(current_balance)

        signals_log.append({
            'date': row['Date'],
            'price': current_price,
            'pred': pred_5m_corrected,
            'trend': trend,
            'final': final_signal
        })

    # Metrics
    results['final_balance'] = current_balance
    results['net_profit'] = current_balance - initial_capital

    if results['confirmed_trades'] > 0:
        results['win_rate'] = (results['wins'] / results['confirmed_trades']) * 100
        returns_series = pd.Series(trades)
        cumulative_returns = (1 + returns_series).cumprod()
        results['total_return'] = (cumulative_returns.iloc[-1] - 1) * 100
        results['sharpe_ratio'] = np.sqrt(252 * 288) * returns_series.mean() / returns_series.std() if returns_series.std() > 0 else 0
    else:
        results['win_rate'] = 0
        results['total_return'] = 0
        results['sharpe_ratio'] = 0

    return results, trades, signals_log, avg_bias, equity_curve

# =============
# MAIN
# =============
if __name__ == "__main__":
    print("=" * 60)
    print("  HYBRID DUAL-TIMEFRAME DEMO (Phase 2 Optimized)")
    print("  Stricter Filters | EMA Alignment | Bias Correction")
    print("=" * 60)

    # ... (Model Loading Logic - Condensed for brevity, functionality maintained)
    # ===== LOAD MODELS =====
    print("\n[1/5] Loading models...")
    try:
        big_model = load_model(BIG_BROTHER_MODEL)
        print(f"  ✓ Big Brother (1H) loaded")
    except Exception as e:
        print(f"  ✗ Big Brother not found: {e}")
        big_model = None

    try:
        little_model = load_model(LITTLE_BROTHER_MODEL)
        print(f"  ✓ Little Brother (5M) loaded")
    except Exception as e:
        print(f"  ✗ Little Brother not found: {e}")
        little_model = None

    if not big_model or not little_model:
        exit()

    with open(SCALER_SMALL, 'rb') as f:
        scaler = pickle.load(f)
    with open(CLOSE_SCALER, 'rb') as f:
        close_scaler = pickle.load(f)
    print("  ✓ Scalers loaded")

    # ===== LOAD DATA =====
    print("\n[2/5] Loading 2025 test data...")
    df_5m = load_and_preprocess(TEST_5M)
    try:
        df_1h = load_and_preprocess(TEST_1H)
    except:
        print(" Using resampled 1H data...")
        pass

    # ===== RUN PREDICTIONS =====
    print("\n[3/5] Running predictions...")

    # Little Brother (5M)
    print("  Running Little Brother (5M)...")
    df_5m_scaled = df_5m.copy()
    df_5m_scaled[FEATURE_COLS] = scaler.transform(df_5m[FEATURE_COLS])

    predict_5m = tf.keras.utils.timeseries_dataset_from_array(
        data=df_5m_scaled[FEATURE_COLS].values, targets=None,
        sequence_length=LOOKBACK_SMALL, batch_size=256, shuffle=False
    )
    pred_5m_scaled = little_model.predict(predict_5m, verbose=0)
    little_predictions = close_scaler.inverse_transform(pred_5m_scaled).flatten()
    print(f"  ✓ {len(little_predictions)} predictions")

    # Big Brother (1H)
    print("  Running Big Brother (1H)...")
    df_1h_scaled = df_1h.copy()
    df_1h_scaled[FEATURE_COLS] = scaler.transform(df_1h[FEATURE_COLS])

    predict_1h = tf.keras.utils.timeseries_dataset_from_array(
        data=df_1h_scaled[FEATURE_COLS].values, targets=None,
        sequence_length=LOOKBACK_BIG, batch_size=64, shuffle=False
    )
    pred_1h_scaled = big_model.predict(predict_1h, verbose=0)
    big_predictions = close_scaler.inverse_transform(pred_1h_scaled).flatten()
    print(f"  ✓ {len(big_predictions)} predictions")

    # ===== HYBRID BACKTEST =====
    print("\n[4/5] Running Phase 2 Optimization & Simulation...")

    metrics, trades, signals, final_bias, equity_curve = hybrid_backtest(
        df_5m, little_predictions,
        df_1h, big_predictions,
        LOOKBACK_SMALL, LOOKBACK_BIG,
        initial_capital=200.0
    )

    # ===== DISPLAY RESULTS =====
    print("\n" + "=" * 60)
    print("  HYBRID BACKTEST RESULTS (OPTIMIZED)")
    print("=" * 60)

    print(f"\n  📊 Signal Analysis:")
    print(f"     Total Candles:      {metrics['total_signals']}")
    print(f"     Confirmed Trades:   {metrics['confirmed_trades']} (Reduced Noise)")
    print(f"     Filtered Signals:   {metrics['filtered_signals']}")

    print(f"\n  💰 Trading Performance:")
    print(f"     BUY Trades:         {metrics['buy_signals']}")
    print(f"     SELL Trades:        {metrics['sell_signals']}")
    print(f"     Wins:               {metrics['wins']}")
    print(f"     Losses:             {metrics['losses']}")
    print(f"     Win Rate:           {metrics['win_rate']:.2f}%")
    print(f"     Total Return:       {metrics['total_return']:.2f}%")

    print(f"\n  💵 Paper Trade Simulation ($200 Start):")
    print(f"     Initial Balance:    ${metrics['initial_balance']:.2f}")
    print(f"     Final Balance:      ${metrics['final_balance']:.2f}")
    profit_sign = "+" if metrics['net_profit'] >= 0 else "-"
    print(f"     Net Profit:         {profit_sign}${abs(metrics['net_profit']):.2f}")
    print("=" * 60)

    print("=" * 60)
    print("\n" + "=" * 60)
    print("  CURRENT TRADING SIGNAL (Bias Corrected)")
    print("=" * 60)
    current_price = df_5m['Close'].iloc[-1]

    # 5M Prediction logic...
    latest_5m = df_5m_scaled.iloc[-LOOKBACK_SMALL:][FEATURE_COLS].values[None, ...]
    pred_5m_raw = close_scaler.inverse_transform(little_model.predict(latest_5m, verbose=0))[0][0]
    target_5m_corrected = pred_5m_raw + final_bias

    # 1H Prediction logic...
    latest_1h = df_1h_scaled.iloc[-LOOKBACK_BIG:][FEATURE_COLS].values[None, ...]
    pred_1h_raw = close_scaler.inverse_transform(big_model.predict(latest_1h, verbose=0))[0][0]
    current_price_1h = df_1h['Close'].iloc[-1]
    ema_20_1h = df_1h['EMA_20'].iloc[-1]
    ema_50_1h = df_1h['EMA_50'].iloc[-1]

    ema_bullish = (current_price_1h > ema_50_1h) and (ema_20_1h > ema_50_1h)
    ema_bearish = (current_price_1h < ema_50_1h) and (ema_20_1h < ema_50_1h)
    pred_bullish_1h = (pred_1h_raw + final_bias) > current_price_1h

    if pred_bullish_1h and ema_bullish:
        trend = "BULLISH (Strong Pattern)"
        trend_icon = "🟢"
    elif not pred_bullish_1h and ema_bearish:
        trend = "BEARISH (Strong Pattern)"
        trend_icon = "🔴"
    else:
        trend = "NEUTRAL (EMA Mismatch)"
        trend_icon = "⚪"

    print(f"\n  [Big Brother] 1H Structure: {trend_icon} {trend}")
    print(f"     Price: ${current_price_1h:.2f} | EMA50: ${ema_50_1h:.2f}")

    move_pct = (target_5m_corrected - current_price) / current_price
    min_thresh = 0.0015
    if move_pct > min_thresh: entry = "BUY"
    elif move_pct < -min_thresh: entry = "SELL"
    else: entry = "WAIT (Weak Move)"

    print(f"\n  [Little Brother] 5M Signal: {entry}")
    print(f"     Current: ${current_price:.2f}")
    print(f"     Target:  ${target_5m_corrected:.2f}")

    print(f"\n  {'─' * 40}")
    if "BULLISH" in trend and entry == "BUY": final_sig = "✅ BUY NOW"
    elif "BEARISH" in trend and entry == "SELL": final_sig = "✅ SELL NOW"
    else: final_sig = "⏸️ WAIT / FILTERED"
    print(f"  >> FINAL SIGNAL: {final_sig}")
    print("=" * 60)

    print("\n  📊 Charts displayed above")

    # Update Fig 1
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.08,
                       subplot_titles=(f'Hybrid Performance ({metrics["win_rate"]:.1f}%)',
                                     f'Account Balance ($200 Start) -> ${metrics["final_balance"]:.2f}',
                                     'Signals'))

    dates = df_5m['Date'].iloc[LOOKBACK_SMALL-1:LOOKBACK_SMALL-1+len(little_predictions)]
    real = df_5m['Close'].iloc[LOOKBACK_SMALL-1:LOOKBACK_SMALL-1+len(little_predictions)]

    fig.add_trace(go.Scatter(x=dates, y=real, mode='lines', name='Price', line=dict(color='#00FF00')), row=1, col=1)

    # Plot equity curve against trade count

    fig.add_trace(go.Scatter(x=list(range(len(equity_curve))), y=equity_curve,
                             mode='lines', name='Balance ($)',
                             line=dict(color='#00BFFF', width=2), fill='tozeroy'), row=2, col=1)

    fig.update_layout(template='plotly_dark', height=900, title='Phase 2 Results + Paper Trade Simulation')
    fig.show()



# OR THIS ALSO THIS PREDICTS THE 2025 STOCK MOVEMENT THE METHOS I USED IS TO SEPARATE THE 2025 DATA SET AND CREATE IT AS TEST VALIDATION

In [ ]:
# =============
# PREDICT 2025 STOCK MOVEMENT (AI Hybrid)
# =============
import numpy as np
import pandas as pd
import pickle
import warnings
import os
import sys
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import load_model

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =============
# CONFIGURATION
# =============
# Adjust paths if models are in a specific directory
BIG_BROTHER_MODEL = 'big_brother_v2.keras'
LITTLE_BROTHER_MODEL = 'little_brother_v2.keras'
SCALER_SMALL = 'scaler_small.pkl'
CLOSE_SCALER = 'close_scaler.pkl'
TEST_5M_FILENAME = 'TEST_5m.csv'
TEST_1H_FILENAME = 'TEST_1h.csv'

LOOKBACK_BIG = 168   # 1 week of hourly data
LOOKBACK_SMALL = 48  # 4 hours of 5-min data

FEATURE_COLS = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Returns', 'EMA_20', 'EMA_50', 'EMA_Diff',
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Width', 'BB_Position', 'ATR', 'Volatility'
]

# =============
# TECHNICAL INDICATORS
# =============
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    return macd, signal_line, histogram

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    middle = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def add_technical_indicators(df):
    df['Returns'] = df['Close'].pct_change()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_Diff'] = df['EMA_20'] - df['EMA_50']
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Volatility'] = df['Returns'].rolling(window=20).std()
    return df

def find_file(filename):
    """Helper to find file in DATA_SET or current directory"""
    # 1. Check DATA_SET/filename
    path1 = os.path.join('DATA_SET', filename)
    if os.path.exists(path1):
        return path1

    # 2. Check ./filename (Colab root)
    path2 = filename
    if os.path.exists(path2):
        return path2

    return None

def load_and_preprocess(filename, filter_year=None):
    filepath = find_file(filename)
    if not filepath:
        print(f"ERROR: File not found: {filename}")
        print(f"  Checked: DATA_SET/{filename} and ./{filename}")
        print(f"  Current Directory: {os.getcwd()}")
        return None

    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, sep=';')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')

    if filter_year:
        print(f"Filtering for Year {filter_year}...")
        df_filtered = df[df['Date'].dt.year == filter_year]
        if df_filtered.empty:
            print(f"WARNING: No data found for year {filter_year} in {filepath}")
            print(f"  Available Date Range: {df['Date'].min()} to {df['Date'].max()}")
            print("  -> Proceeding with ALL data in file (assuming this is the intended test set).")
        else:
            print(f"Found {len(df_filtered)} rows for {filter_year}")
            df = df_filtered

    df = df.sort_values('Date').reset_index(drop=True)
    df = add_technical_indicators(df)
    df = df.dropna().reset_index(drop=True)
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('float32')
    return df

# =============
# HYBRID BACKTEST LOGIC
# =============
def hybrid_backtest(df_5m, little_predictions, df_1h, big_predictions, lookback_5m, lookback_1h):
    results = {
        'total_signals': 0, 'confirmed_trades': 0, 'wins': 0, 'losses': 0,
        'buy_signals': 0, 'sell_signals': 0, 'filtered_signals': 0
    }

    trades = []

    # --- Bias Correction Setup ---
    bias_window = []
    BIAS_WINDOW_SIZE = 50

    # --- Filters ---
    RSI_BUY_MAX = 70
    RSI_SELL_MIN = 30
    MIN_THRESHOLD = 0.0015

    # Align Data
    # Ensure indices match the predictions length
    df_5m_subset = df_5m.iloc[lookback_5m-1 : lookback_5m-1+len(little_predictions)].copy()
    df_5m_subset['Predicted_Raw'] = little_predictions

    df_1h_subset = df_1h.iloc[lookback_1h-1 : lookback_1h-1+len(big_predictions)].copy()
    df_1h_subset['Predicted_Raw'] = big_predictions

    # 1H Mapping
    df_1h_subset['Hour'] = df_1h_subset['Date'].dt.floor('H')
    hourly_trend = df_1h_subset.set_index('Hour')[
        ['Close', 'Predicted_Raw', 'EMA_20', 'EMA_50']
    ].to_dict('index')

    print("\nSimulating Trading Logic on Data...")

    for i in range(len(df_5m_subset) - 1):
        row = df_5m_subset.iloc[i]
        current_price = row['Close']
        actual_next = df_5m_subset.iloc[i + 1]['Close']

        # 1. Update Bias
        raw_pred_5m = row['Predicted_Raw']
        current_bias = current_price - raw_pred_5m
        bias_window.append(current_bias)
        if len(bias_window) > BIAS_WINDOW_SIZE:
            bias_window.pop(0)
        avg_bias = sum(bias_window) / len(bias_window)

        pred_5m_corrected = raw_pred_5m + avg_bias

        # 2. Big Brother Trend
        hour_key = row['Date'].floor('H')
        trend = "NEUTRAL"

        if hour_key in hourly_trend:
            h_data = hourly_trend[hour_key]
            pred_bullish = (h_data['Predicted_Raw'] - h_data['Close']) > -avg_bias
            ema_bullish = (h_data['Close'] > h_data['EMA_50']) and (h_data['EMA_20'] > h_data['EMA_50'])
            ema_bearish = (h_data['Close'] < h_data['EMA_50']) and (h_data['EMA_20'] < h_data['EMA_50'])

            if pred_bullish and ema_bullish:
                trend = "BULLISH"
            elif not pred_bullish and ema_bearish:
                trend = "BEARISH"

        # 3. Little Brother Entry
        pred_change = (pred_5m_corrected - current_price) / current_price

        entry_signal = "WAIT"
        if pred_change > MIN_THRESHOLD:
            entry_signal = "BUY"
        elif pred_change < -MIN_THRESHOLD:
            entry_signal = "SELL"

        # 4. RSI Filter
        if entry_signal == "BUY" and row['RSI'] > RSI_BUY_MAX:
            entry_signal = "WAIT"
            results['filtered_signals'] += 1
        if entry_signal == "SELL" and row['RSI'] < RSI_SELL_MIN:
            entry_signal = "WAIT"
            results['filtered_signals'] += 1

        # === FINAL DECISION ===
        results['total_signals'] += 1

        if trend == "BULLISH" and entry_signal == "BUY":
            results['buy_signals'] += 1
            results['confirmed_trades'] += 1
            if actual_next > current_price:
                results['wins'] += 1
            else:
                results['losses'] += 1
            trades.append((actual_next - current_price) / current_price)

        elif trend == "BEARISH" and entry_signal == "SELL":
            results['sell_signals'] += 1
            results['confirmed_trades'] += 1
            if actual_next < current_price:
                results['wins'] += 1
            else:
                results['losses'] += 1
            trades.append((current_price - actual_next) / current_price)

    # Metrics
    if results['confirmed_trades'] > 0:
        results['win_rate'] = (results['wins'] / results['confirmed_trades']) * 100
    else:
        results['win_rate'] = 0

    return results, trades

# =============
# MAIN
# =============
if __name__ == "__main__":
    print("=" * 60)
    print("  2025 STOCK PREDICTION & ANALYSIS")
    print("=" * 60)

    def check_model_exists(name):
        if os.path.exists(name): return True
        return False

    if not check_model_exists(BIG_BROTHER_MODEL) or not check_model_exists(LITTLE_BROTHER_MODEL):
        print("CRITICAL ERROR: Models not found in current directory.")
        print(f"Looking for: {BIG_BROTHER_MODEL} and {LITTLE_BROTHER_MODEL}")
        print("Please upload the .keras model files.")
        sys.exit(1)

    # Load artifacts
    print("Loading models and scalers...")
    try:
        big_model = load_model(BIG_BROTHER_MODEL)
        little_model = load_model(LITTLE_BROTHER_MODEL)

        with open(SCALER_SMALL, 'rb') as f:
            scaler = pickle.load(f)
        with open(CLOSE_SCALER, 'rb') as f:
            close_scaler = pickle.load(f)
    except Exception as e:
        print(f"Error loading models/scalers: {e}")
        sys.exit(1)

    # Load 2025 Data
    print("\nLoading Data for 2025...")
    df_5m = load_and_preprocess(TEST_5M_FILENAME, filter_year=2025)
    df_1h = load_and_preprocess(TEST_1H_FILENAME, filter_year=2025)

    if df_5m is None or df_5m.empty or df_1h is None or df_1h.empty:
        print("\nCRITICAL ERROR: Could not proceed due to missing or empty data.")
        print("Please check that TEST_5m.csv and TEST_1h.csv are uploaded.")
        sys.exit(1)

    # Predictions
    print("\nGenerating Predictions...")

    # 5M
    df_5m_scaled = df_5m.copy()
    df_5m_scaled[FEATURE_COLS] = scaler.transform(df_5m[FEATURE_COLS])
    predict_5m = tf.keras.utils.timeseries_dataset_from_array(
        data=df_5m_scaled[FEATURE_COLS].values, targets=None,
        sequence_length=LOOKBACK_SMALL, batch_size=256, shuffle=False
    )
    pred_5m_scaled = little_model.predict(predict_5m, verbose=0)
    little_predictions = close_scaler.inverse_transform(pred_5m_scaled).flatten()

    # 1H
    df_1h_scaled = df_1h.copy()
    df_1h_scaled[FEATURE_COLS] = scaler.transform(df_1h[FEATURE_COLS])
    predict_1h = tf.keras.utils.timeseries_dataset_from_array(
        data=df_1h_scaled[FEATURE_COLS].values, targets=None,
        sequence_length=LOOKBACK_BIG, batch_size=64, shuffle=False
    )
    pred_1h_scaled = big_model.predict(predict_1h, verbose=0)
    big_predictions = close_scaler.inverse_transform(pred_1h_scaled).flatten()

    # Run Compare
    results, trades = hybrid_backtest(
        df_5m, little_predictions,
        df_1h, big_predictions,
        LOOKBACK_SMALL, LOOKBACK_BIG
    )

    print("\n" + "=" * 60)
    print("  2025 PREDICTION RESULTS")
    print("=" * 60)
    print(f"  Total Opportunities Scanned: {results['total_signals']}")
    print(f"  Trades Taken (Agreement):    {results['confirmed_trades']}")
    print(f"  Win Rate:                    {results['win_rate']:.2f}%")
    print(f"  Wins:                        {results['wins']}")
    print(f"  Losses:                      {results['losses']}")
    print("=" * 60)

    if len(trades) > 0:
        total_return_pct = (np.prod([1+t for t in trades]) - 1) * 100
        print(f"  Estimated Return:            {total_return_pct:.2f}% (Compounded)")

    # =============
    # VISUALIZATION
    # =============
    print("\nGenerating Graph...")
    start_idx = LOOKBACK_SMALL - 1
    end_idx = start_idx + len(little_predictions)

    plot_dates = df_5m['Date'].iloc[start_idx:end_idx]
    real_price = df_5m['Close'].iloc[start_idx:end_idx]

    # Create Figure
    fig = make_subplots(rows=1, cols=1, subplot_titles=("2025: Real Market vs AI Prediction"))

    # 1. Real Data (Green)
    fig.add_trace(go.Scatter(
        x=plot_dates,
        y=real_price,
        mode='lines',
        name='Real Price (Market)',
        line=dict(color='#00FF00', width=1)
    ))

    # 2. AI Prediction (Red)
    fig.add_trace(go.Scatter(
        x=plot_dates,
        y=little_predictions,
        mode='lines',
        name='AI Prediction',
        line=dict(color='#FF0000', width=1, dash='dot')
    ))

    fig.update_layout(
        template='plotly_dark',
        title_text='AI Model Performance: Real (Green) vs Predicted (Red)',
        xaxis_title='Date',
        yaxis_title='Price',
        height=600,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Show figure
    print("Graph generated. Displaying...")
    fig.show()


# TEST DEMO TO PREDICT THE 2026 XAU/USD MARKET MOVEMENT

In [ ]:
# =============
# SIMULATE 2026 WEALTH (Monte Carlo Projection)
# =============
import numpy as np
import pandas as pd
import pickle
import warnings
import os
import sys
import random
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import load_model

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =============
# CONFIGURATION
# =============
BIG_BROTHER_MODEL = 'big_brother_v2.keras'
LITTLE_BROTHER_MODEL = 'little_brother_v2.keras'
SCALER_SMALL = 'scaler_small.pkl'
CLOSE_SCALER = 'close_scaler.pkl'

# Filenames to check
TEST_5M_FILENAME = 'TEST_5m.csv'
TEST_1H_FILENAME = 'TEST_1h.csv'

# BACKTEST SETTINGS
LOOKBACK_BIG = 168
LOOKBACK_SMALL = 48
INITIAL_CAPITAL = 200.0  # User Request

FEATURE_COLS = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'Returns', 'EMA_20', 'EMA_50', 'EMA_Diff',
    'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist',
    'BB_Width', 'BB_Position', 'ATR', 'Volatility'
]

# =============
# HELPER FUNCTIONS
# =============
# ... (Technical indicators reduced for brevity, assuming data has them or we calculate same as before)
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    histogram = macd - signal_line
    return macd, signal_line, histogram

def calculate_bollinger_bands(prices, period=20, std_dev=2):
    middle = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    return upper, middle, lower

def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window=period).mean()

def add_technical_indicators(df):
    df['Returns'] = df['Close'].pct_change()
    df['EMA_20'] = df['Close'].ewm(span=20, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_Diff'] = df['EMA_20'] - df['EMA_50']
    df['RSI'] = calculate_rsi(df['Close'], 14)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Volatility'] = df['Returns'].rolling(window=20).std()
    return df

def find_file(filename):
    path1 = os.path.join('DATA_SET', filename)
    if os.path.exists(path1): return path1
    if os.path.exists(filename): return filename
    return None

def load_and_preprocess(filename, filter_year=None):
    filepath = find_file(filename)
    if not filepath:
        print(f"Skipping {filename} (not found)")
        return None
    print(f"Loading {filepath}...")
    df = pd.read_csv(filepath, sep=';')
    df['Date'] = pd.to_datetime(df['Date'], format='%Y.%m.%d %H:%M')

    if filter_year:
        df_filtered = df[df['Date'].dt.year == filter_year]
        if not df_filtered.empty:
            df = df_filtered

    df = df.sort_values('Date').reset_index(drop=True)
    df = add_technical_indicators(df)
    df = df.dropna().reset_index(drop=True)
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('float32')
    return df

def hybrid_backtest(df_5m, little_predictions, df_1h, big_predictions):
    # Returns list of trade pct returns
    trades = []

    # Simple filters (matching predict_2025.py)
    RSI_BUY_MAX = 70
    RSI_SELL_MIN = 30
    MIN_THRESHOLD = 0.0015
    BIAS_WINDOW_SIZE = 50
    bias_window = []

    # Align
    df_5m_subset = df_5m.iloc[LOOKBACK_SMALL-1 : LOOKBACK_SMALL-1+len(little_predictions)].copy()
    df_5m_subset['Predicted_Raw'] = little_predictions

    df_1h_subset = df_1h.iloc[LOOKBACK_BIG-1 : LOOKBACK_BIG-1+len(big_predictions)].copy()
    df_1h_subset['Predicted_Raw'] = big_predictions
    df_1h_subset['Hour'] = df_1h_subset['Date'].dt.floor('H')
    hourly_trend = df_1h_subset.set_index('Hour')[['Close', 'Predicted_Raw', 'EMA_20', 'EMA_50']].to_dict('index')

    for i in range(len(df_5m_subset) - 1):
        row = df_5m_subset.iloc[i]
        current_price = row['Close']
        actual_next = df_5m_subset.iloc[i + 1]['Close']

        # Bias
        raw_pred_5m = row['Predicted_Raw']
        current_bias = current_price - raw_pred_5m
        bias_window.append(current_bias)
        if len(bias_window) > BIAS_WINDOW_SIZE: bias_window.pop(0)
        avg_bias = sum(bias_window) / len(bias_window)
        pred_5m_corrected = raw_pred_5m + avg_bias

        # Trend
        hour_key = row['Date'].floor('H')
        trend = "NEUTRAL"
        if hour_key in hourly_trend:
            h_data = hourly_trend[hour_key]
            pred_bullish = (h_data['Predicted_Raw'] - h_data['Close']) > -avg_bias
            ema_bullish = (h_data['Close'] > h_data['EMA_50']) and (h_data['EMA_20'] > h_data['EMA_50'])
            ema_bearish = (h_data['Close'] < h_data['EMA_50']) and (h_data['EMA_20'] < h_data['EMA_50'])
            if pred_bullish and ema_bullish: trend = "BULLISH"
            elif not pred_bullish and ema_bearish: trend = "BEARISH"

        # Entry
        pred_change = (pred_5m_corrected - current_price) / current_price
        entry_signal = "WAIT"
        if pred_change > MIN_THRESHOLD: entry_signal = "BUY"
        elif pred_change < -MIN_THRESHOLD: entry_signal = "SELL"

        # RSI
        if entry_signal == "BUY" and row['RSI'] > RSI_BUY_MAX: entry_signal = "WAIT"
        if entry_signal == "SELL" and row['RSI'] < RSI_SELL_MIN: entry_signal = "WAIT"

        # Execute
        if trend == "BULLISH" and entry_signal == "BUY":
            trades.append((actual_next - current_price) / current_price)
        elif trend == "BEARISH" and entry_signal == "SELL":
            trades.append((current_price - actual_next) / current_price)

    return trades

def monte_carlo_simulation(trades_history, num_simulations=50, years=2, initial_balance=2000.0)

    trades_per_year = len(trades_history)
    total_trades_future = trades_per_year * years

    simulation_results = []

    for _ in range(num_simulations):
        balance = initial_balance
        equity_curve = [balance]

        for _ in range(total_trades_future):
            r_trade = random.choice(trades_history)

            balance = balance * (1 + r_trade)
            equity_curve.append(balance)

        simulation_results.append(equity_curve)

    return simulation_results, total_trades_future

# =============
# MAIN
# =============
if __name__ == "__main__":
    print("=" * 60)
    print("  WEALTH SIMULATION (2026)")
    print(f"  Starting Capital: ${INITIAL_CAPITAL}")
    print("=" * 60)

    # 1. Load Models & Data (2025 Baseline)
    try:
        big_model = load_model(BIG_BROTHER_MODEL)
        little_model = load_model(LITTLE_BROTHER_MODEL)
        with open(SCALER_SMALL, 'rb') as f: scaler = pickle.load(f)
        with open(CLOSE_SCALER, 'rb') as f: close_scaler = pickle.load(f)
    except:
        print("Error loading models. Make sure .keras and .pkl files are present.")
        sys.exit(1)

    df_5m = load_and_preprocess(TEST_5M_FILENAME)
    df_1h = load_and_preprocess(TEST_1H_FILENAME)

    if df_5m is None or df_1h is None:
        print("Data missing. Cannot benchmark.")
        sys.exit(1)

    # 2. Run Baseline Backtest
    print("\nBenchmarking on Test Data to map probability...")

    # Predict 5M
    df_5m_s = df_5m.copy()
    df_5m_s[FEATURE_COLS] = scaler.transform(df_5m[FEATURE_COLS])
    p_5m = little_model.predict(tf.keras.utils.timeseries_dataset_from_array(
        df_5m_s[FEATURE_COLS].values, None, LOOKBACK_SMALL, batch_size=256, shuffle=False
    ), verbose=0)
    pred_5m = close_scaler.inverse_transform(p_5m).flatten()

    # Predict 1H
    df_1h_s = df_1h.copy()
    df_1h_s[FEATURE_COLS] = scaler.transform(df_1h[FEATURE_COLS])
    p_1h = big_model.predict(tf.keras.utils.timeseries_dataset_from_array(
        df_1h_s[FEATURE_COLS].values, None, LOOKBACK_BIG, batch_size=64, shuffle=False
    ), verbose=0)
    pred_1h = close_scaler.inverse_transform(p_1h).flatten()

    # Extract Trades
    historical_trades = hybrid_backtest(df_5m, pred_5m, df_1h, pred_1h)

    if not historical_trades:
        print("No trades found in baseline data. Cannot simulate.")
        sys.exit()

    win_rate = len([x for x in historical_trades if x > 0]) / len(historical_trades)
    avg_return = np.mean(historical_trades) * 100
    print(f"  Baseline Metrics:")
    print(f"  - Win Rate: {win_rate*100:.1f}%")
    print(f"  - Avg Return/Trade: {avg_return:.2f}%")
    print(f"  - Total Samples: {len(historical_trades)}")

    # 3. Simulate Future (Monte Carlo)
    print("\nSimulating 2026 (Monte Carlo Projection)...")
    sim_curves, num_future_trades = monte_carlo_simulation(historical_trades, num_simulations=50, years=1, initial_balance=INITIAL_CAPITAL)

    # Calculate Percentiles
    # Transpose to get distribution at each step
    steps = len(sim_curves[0])
    median_curve = []
    p90_curve = []
    p10_curve = []

    for i in range(steps):
        step_vals = [curve[i] for curve in sim_curves]
        median_curve.append(np.median(step_vals))
        p90_curve.append(np.percentile(step_vals, 90))
        p10_curve.append(np.percentile(step_vals, 10))

    final_median = median_curve[-1]
    final_p90 = p90_curve[-1]
    final_p10 = p10_curve[-1]

    print("\n" + "=" * 60)
    print("  SIMULATION RESULTS (End of 2026)")
    print("=" * 60)
    print(f"  Starting Balance:   ${INITIAL_CAPITAL:,.2f}")
    print(f"  Projected Median:   ${final_median:,.2f} (Likely)")
    print(f"  Optimistic (90%):   ${final_p90:,.2f} (Lucky Streak)")
    print(f"  Pessimistic (10%):  ${final_p10:,.2f} (Bad Streak)")
    print("=" * 60)

    # 4. Visualization
    fig = make_subplots(rows=1, cols=1, subplot_titles=(f"Wealth Projection 2026 (Start: ${INITIAL_CAPITAL})"))

    # Plot all simulation traces (faint)
    x_axis = list(range(steps))
    for curve in sim_curves[:20]: # Show first 20 lines
        fig.add_trace(go.Scatter(x=x_axis, y=curve, mode='lines', line=dict(color='rgba(255, 255, 255, 0.1)'), showlegend=False))

    # Plot Median (Green - "Real" expectation)
    fig.add_trace(go.Scatter(x=x_axis, y=median_curve, mode='lines', name='Expected Growth (Median)', line=dict(color='#00FF00', width=3)))

    # Plot Bands (Red - "Risk/Variance")
    fig.add_trace(go.Scatter(x=x_axis, y=p90_curve, mode='lines', name='Optimistic Top', line=dict(color='#FF0000', dash='dot')))
    fig.add_trace(go.Scatter(x=x_axis, y=p10_curve, mode='lines', name='Pessimistic Bottom', line=dict(color='#FF0000', dash='dot')))

    fig.update_layout(
        template='plotly_dark',
        title='Projected Account Balance (2026)',
        xaxis_title='Trade Count (Approx 1 Year)',
        yaxis_title='Balance ($)',
        height=600
    )

    print("Displaying Graph...")
    fig.show()
